<a href="https://colab.research.google.com/github/rockett-m/CS0135/blob/main/hw2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [131]:
"""
A coding assignment that combines splitting data into train, test, and validation sets using scikit-learn (sklearn):

Problem Statement:

You are given a dataset containing information about different types of iris flowers (sepal length, sepal width,
petal length, and petal width). Your task is to build a machine learning model to classify the iris flowers into
three different classes (setosa, versicolor, virginica) based on their physical characteristics.

You are to do the following:
Step 1: Load the iris dataset
    Use the following code to load the iris dataset into a pandas dataframe:

Step 2: Split the data into training, validation, and test sets
    Use the following code to split the data into training (60%), validation (20%), and test (20%) sets:

Step 3: Train a classifier (this function is provided for you)
    Use the following code to train a support vector machine (SVM) classifier on the training set:

Step 4: Evaluate the model on the validation set
    Use the following code to evaluate the performance of the trained classifier on the validation set:
    from sklearn.metrics import accuracy_score

Step 5: Evaluate the model on the test set
    Use the following code to evaluate the performance of the trained classifier on the test set:

This assignment is designed to help you understand the basics of splitting data into train, validation, and test sets,
as well as training a simple machine learning model using scikit-learn. The assignment can be extended by trying
different classifiers, tuning the parameters, or exploring the dataset further.

Step 6: Then, do the same for K-Fold cross validation.
"""
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.model_selection import KFold
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

def load_data():
    """
    Load the iris dataset and return it as a pandas dataframe

    Returns:
        df_out (pandas dataframe): The iris dataset
    """
    from sklearn.datasets import load_iris

    #TODO load data, convert to dataframe, and return
    iris = load_iris()
    df_out = pd.DataFrame(data=iris.data, columns=iris.feature_names)
    df_out['target'] = iris.target

    return df_out


def split_data(df_in):
    """
    Split the data into training, validation, and test sets

    Parameters:
    df_in (pandas dataframe): The iris dataset

    Returns:
    X_train_out (numpy array): Training set features
    X_val_out   (numpy array): Validation set features
    X_test_out  (numpy array): Test set features
    y_train_out (numpy array): Training set targets
    y_val_out   (numpy array): Validation set targets
    y_test_out  (numpy array): Test set targets
    """
    # TODO split train and test, and then split train and val

    # test data does not change! (20%)
    # 80% to training
    # take 20% of training data into val data
    # call split twice !!!

    # 150, 4 full df in
    # 120, 4 train dims [0:119, 4] # first 120 rows of df_in
    #  30, 4 val dims   [90:119, 4] # last 30 rows of train dims
    #  90, 4 train dims [0:89, 4]  # after div into train and val these are final train dims
    #  30, 4 test dims (lock it away!) [120:149, 4] # last 30 rows of df_in

    # X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.33, random_state=42)
    # X_train, X_test, y_train, y_test = train_test_split(X= , y= , test_size=0.8)

    # separate X and y from dataframe
    X, y = df_in.iloc[:,:4], df_in['target']
    # send 80% to train and 20% to test
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=0.80) 
    # 25% of (80% df in train) to get 20% total for val
    X_train_out, X_val_out, y_train_out, y_val_out = train_test_split(X_train, y_train, train_size=0.75)

    return X_train_out, X_val_out, X_test, y_train_out, y_val_out, y_test


def train_classifier(X_train_in, y_train_in):
    """
    Train a support vector machine (SVM) classifier on the training set

    Parameters:
    X_train_in (numpy array): Training set features
    y_train_in (numpy array): Training set targets

    Returns:
    clf_out (SVM classifier): Trained SVM classifier
    """
    clf_out = SVC(kernel='linear', C=1, random_state=0)
    clf_out.fit(X_train_in, y_train_in)

    return clf_out


def evaluate_model(clf, X, y):
    """
    Evaluate the performance of the trained classifier on a given set

    Parameters:
    clf (SVM classifier): Trained SVM classifier
    X (numpy array): Features of the set to evaluate the classifier on
    y (numpy array): Targets of the set to evaluate the classifier on

    Returns:
    accuracy_out (float): Accuracy of the classifier on the given set
    """
    # TODO evaluate the model on data
    accuracy_test_out = clf.score(X, y)

    return accuracy_test_out


def k_fold_cross_validation(df, k=5):
    """
    Perform k-fold cross validation on the iris dataset

    Parameters:
    df (pandas dataframe): The iris dataset
    k (int, optional): The number of folds. Default is 5.

    Returns:
    accuracy_out (list): A list of accuracy_out scores for each fold
    """
    from sklearn.model_selection import cross_val_score
    from sklearn.model_selection import KFold
    # TODO do 5-fold on dataset and return the accuracy across the five folds
    # X=iris.data, y=iris.target
    X, y = df.iloc[:,:4].values, df['target']
    clf_out = SVC(kernel='linear', C=1, random_state=0)

    # (n_splits=5, *, shuffle=False, random_state=None)
    accuracy_five_folds = []
    kf = KFold(n_splits=k) # k == n_splits

    for train_index, test_index in kf.split(X):
        # print("TRAIN:", train_index, "TEST:", test_index)
        X_train, X_test = X[train_index], X[test_index]
        y_train, y_test = y[train_index], y[test_index]

        clf = train_classifier(X_train, y_train)
        accuracy_val = evaluate_model(clf, X_test, y_test)
        accuracy_five_folds.append(accuracy_val)

    # accuracy_five_folds = cross_val_score(clf_out, X=iris.data, y=iris.target, cv=k, scoring="accuracy")
    # accuracy_five_folds = cross_val_score(clf_out, X, y, cv=k, scoring="accuracy")

    return accuracy_five_folds



if __name__ == '__main__':
    # Load the iris dataset
    df_in = load_data()

    # Train, Val, Test
    # Split the data into training, validation, and test sets
    X_train, X_val, X_test, y_train, y_val, y_test = split_data(df_in)

    # Train a classifier
    clf = train_classifier(X_train, y_train)

    # Evaluate the model on the validation set
    accuracy_val = evaluate_model(clf, X_val, y_val)
    print("Accuracy on validation set:", accuracy_val)

    # Evaluate the model on the test set
    accuracy_test = evaluate_model(clf, X_test, y_test)
    print("Accuracy on test set:", accuracy_test)

    # K-FOLD
    # Perform k-fold cross validation
    accuracy = k_fold_cross_validation(df_in)

    # Print the accuracy scores for each fold
    for idx, acc_score in enumerate(accuracy):
        print("Accuracy on fold", idx + 1, ":", acc_score)

Accuracy on validation set: 0.9333333333333333
Accuracy on test set: 1.0
Accuracy on fold 1 : 1.0
Accuracy on fold 2 : 1.0
Accuracy on fold 3 : 0.8666666666666667
Accuracy on fold 4 : 1.0
Accuracy on fold 5 : 0.8666666666666667


Attribute Information:

sepal length in cm
sepal width in cm
petal length in cm
petal width in cm
class:
-- Iris Setosa
-- Iris Versicolour
-- Iris Virginica
https://archive.ics.uci.edu/ml/datasets/iris

'feature_names' = list ['sepal length (cm)', 'sepal width (cm)', 'petal length (cm)', 'petal width (cm)']

'target_names' = array(['setosa', 'versicolor', 'virginica'],

In [ ]:
'''
def view_dataset(debug=False, verbose=False):
    """
    Load the iris dataset and return it as a pandas dataframe

    Returns:
        df_out (pandas dataframe): The iris dataset
    """
    from sklearn.datasets import load_iris

    #TODO load data, convert to dataframe, and return
    iris = load_iris()

    if verbose:
        print(f'{type(iris) = }, {iris.items = }, {len(iris) = }')

        for elem in iris:
            print(f'{elem = } {iris[elem] = }')

    # print(f'{iris.data = }')
    # print(f'{iris.data = }')
    data = iris.data # (150, 4)
    target = iris.target # (150,)
    target_names = iris.target_names # (3,)
    feature_names = iris.feature_names

    df_data = pd.DataFrame(data)

    if debug:

        print(f'{df_data.head() = }')
        print(f'{df_data.describe() = }')

    if verbose:
        print(f'{df_data = }')

    # df_data = pd.DataFrame(data=iris.data)

    # test data does not change! (20%)
    # 80% to training
    # take 20% of training data into val data
    # call split twice !!!
    # test = df_data[len(df_data):]  # 80% to the end

    # X - data
    # X_data = iris.data[:][:]
    # Y_data = iris.data[:][1]
    # Y - target
    # target

    # iris
    # iris.info()
    # print(iris)
    # df = pd.read()
    # return df_data
    # pass
    """
    Classes: 3
    Samples per class: 50
    Samples total: 150
    Dimensionality: 4
    Features: real, positive
    """
view_dataset(debug=True, verbose=False)
'''

df_data.head() =      0    1    2    3
0  5.1  3.5  1.4  0.2
1  4.9  3.0  1.4  0.2
2  4.7  3.2  1.3  0.2
3  4.6  3.1  1.5  0.2
4  5.0  3.6  1.4  0.2
df_data.describe() =                 0           1           2           3
count  150.000000  150.000000  150.000000  150.000000
mean     5.843333    3.057333    3.758000    1.199333
std      0.828066    0.435866    1.765298    0.762238
min      4.300000    2.000000    1.000000    0.100000
25%      5.100000    2.800000    1.600000    0.300000
50%      5.800000    3.000000    4.350000    1.300000
75%      6.400000    3.300000    5.100000    1.800000
max      7.900000    4.400000    6.900000    2.500000
